# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.5685e19,"""12225729162166314001""","""../../../../data/SampleHDS.jso…","""51981535217579487""","""10734833795674794279""","""16221475036607966456""",false
6.3949e18,"""10858144133816846025""","""../../../../data/SampleHDS.jso…","""16371167012349695065""","""10734833795674794279""","""3274820548361335149""",true
1.7483e19,"""8666476490711445212""","""../../../../data/SampleHDS.jso…","""12342636491773362689""","""14679932339064961627""","""16221475036607966456""",false
1.7024e18,"""7354403747816733489""","""../../../../data/SampleHDS.jso…","""16371167012349695065""","""3009009646360993877""","""16221475036607966456""",false
8.7174e17,"""12258424551632205397""","""../../../../data/SampleHDS.jso…","""51981535217579487""","""3009009646360993877""","""4135340674029727270""",true
2.3170e17,"""6396382143874511791""","""../../../../data/SampleHDS.jso…","""12342636491773362689""","""14035086589861628279""",null,false
1.8433e19,"""18001800542168006""","""../../../../data/SampleHDS.jso…","""12342636491773362689""","""3009009646360993877""","""4135340674029727270""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.4972e19,"""17422485123997285017""","""../../../../data/SampleHDS.jso…","""2138003491835027148""","""FirstMortgage30yr""","""5204744201502184781""","""Rejected"""
4.2698e18,"""1573862126604267032""","""../../../../data/SampleHDS.jso…","""7508068202029349532""","""FirstMortgage30yr""","""12316935218126645863""","""Accepted"""
1.4331e18,"""4507694027456499642""","""../../../../data/SampleHDS.jso…","""17881602612806332164""","""MoneyMarketSavingsAccount""","""5204744201502184781""","""Rejected"""
6.0648e18,"""9385107858368052372""","""../../../../data/SampleHDS.jso…","""7508068202029349532""","""BasicChecking""","""5204744201502184781""","""Rejected"""
5.8943e18,"""6009163988129042585""","""../../../../data/SampleHDS.jso…","""2138003491835027148""","""BasicChecking""","""16399429538149167714""","""Accepted"""
3.0495e18,"""17400663458862046337""","""../../../../data/SampleHDS.jso…","""17881602612806332164""","""UPlusFinPersonal""",null,"""Rejected"""
1.3523e19,"""876426660377517334""","""../../../../data/SampleHDS.jso…","""17881602612806332164""","""BasicChecking""","""16399429538149167714""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
6.0000e18,"""17010288382974320290""","""../../../../data/SampleHDS.jso…","""928003177903087711""","""FirstMortgage30yr""","""5195314161035972811""","""Rejected"""
5.8879e18,"""16627535476138994475""","""../../../../data/SampleHDS.jso…","""8641606584800968118""","""FirstMortgage30yr""","""11484570070419202773""","""Accepted"""
1.3162e19,"""11054302521963209580""","""../../../../data/SampleHDS.jso…","""11795014513929145453""","""MoneyMarketSavingsAccount""","""5195314161035972811""","""Rejected"""
6.8958e18,"""2690860086922430879""","""../../../../data/SampleHDS.jso…","""8641606584800968118""","""BasicChecking""","""5195314161035972811""","""Rejected"""
1.5988e19,"""2483174824290025401""","""../../../../data/SampleHDS.jso…","""928003177903087711""","""BasicChecking""","""5440717972691627548""","""Accepted"""
1.2533e19,"""7427967276565884596""","""../../../../data/SampleHDS.jso…","""11795014513929145453""","""UPlusFinPersonal""",null,"""Rejected"""
6.8039e18,"""17204486796614990752""","""../../../../data/SampleHDS.jso…","""11795014513929145453""","""BasicChecking""","""5440717972691627548""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
3.4990e18,"""12133085546794794703""","""16713742105317773808""","""FirstMortgage30yr""","""12915145627544144786""","""Rejected"""
1.8417e19,"""15839720641583736446""","""18418823717113333765""","""FirstMortgage30yr""","""14619991112982357529""","""Accepted"""
1.0755e18,"""2113161723567041340""","""12841435627529050742""","""MoneyMarketSavingsAccount""","""12915145627544144786""","""Rejected"""
7.1356e18,"""15914827358111653676""","""18418823717113333765""","""BasicChecking""","""12915145627544144786""","""Rejected"""
1.4449e17,"""13183820715536848405""","""16713742105317773808""","""BasicChecking""","""12193564803628093089""","""Accepted"""
8.9969e18,"""7685362011662894839""","""12841435627529050742""","""UPlusFinPersonal""",null,"""Rejected"""
1.7611e19,"""10089310986746519408""","""12841435627529050742""","""BasicChecking""","""12193564803628093089""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'